# Retail Store 데이터 셋 생성
아래의 Git에서 인공적으로 생성된 데이터 세트를 사용 함.

생성된 데이터 세트를 retail_stores_items.csv 저장 했습니다. 
- [원본 Retail Demo Store Git Repo](https://github.com/aws-samples/retail-demo-store)

# 0. 선수 사항
- 이 노트북은 "Amazon Translate" 서비스를 이용합니다. 그래서 이 노트북을 실행하는 Role 에 "TranslateFullAccess" Permission 이 추가 되어져야 합니다.

# 1. 데이터 로딩

In [2]:
import boto3

translate_client = boto3.client(service_name='translate', region_name='us-east-1', use_ssl=True)



In [3]:
import pandas as pd

raw = pd.read_csv("retail_stores_items.csv")
print(raw.columns)
raw.head(3)


Index(['id', 'url', 'sk', 'name', 'category', 'style', 'description',
       'aliases', 'price', 'image', 'gender_affinity', 'current_stock',
       'featured'],
      dtype='object')


,id,url,sk,name,category,style,description,aliases,price,image,gender_affinity,current_stock,featured
0,e1669081-8ffc-4dec-97a6-e9176d7f6651,http://d32da96qlo1y4g.cloudfront.net/#/product...,NaN,Sans Pareil Scarf,apparel,scarf,Sans pareil scarf for women,NaN,124.99,http://d32da96qlo1y4g.cloudfront.net/images/ap...,F,12,NaN
1,cfafd627-7d6b-43a5-be05-4c7937be417d,http://d32da96qlo1y4g.cloudfront.net/#/product...,NaN,Chef Knife,housewares,kitchen,A must-have for your kitchen,NaN,57.99,http://d32da96qlo1y4g.cloudfront.net/images/ho...,NaN,9,NaN
2,6e6ad102-7510-4a02-b8ce-5a0cd6f431d1,http://d32da96qlo1y4g.cloudfront.net/#/product...,NaN,Gainsboro Jacket,apparel,jacket,This gainsboro jacket for women is perfect for...,NaN,133.99,http://d32da96qlo1y4g.cloudfront.net/images/ap...,F,13,NaN


# 2. 데이터 전처리

## 주요 컬럼만 유지하고 삭제

In [4]:
def preprocess_data(raw):
    def create_sinlge_category(s):
        category = s[2]
        style = s[3]        
        single_category = f"{category}|{style}"
        # print(f"{category}|{style}")
        
        return single_category
    
    df = raw.copy()
    df = df.drop(columns=['url','sk','aliases', 'price', 'image', 'gender_affinity', 'current_stock', 'featured'])
    
    df['single_category'] = df.apply(create_sinlge_category, axis=1)
    df = df.drop(columns=['style','category'])
    # df = df.dropna(axis=0, how='any')
    # df = df.reset_index(drop=True)
    return df

df_en = preprocess_data(raw)
df_en.head(3)

,id,name,description,single_category
0,e1669081-8ffc-4dec-97a6-e9176d7f6651,Sans Pareil Scarf,Sans pareil scarf for women,apparel|scarf
1,cfafd627-7d6b-43a5-be05-4c7937be417d,Chef Knife,A must-have for your kitchen,housewares|kitchen
2,6e6ad102-7510-4a02-b8ce-5a0cd6f431d1,Gainsboro Jacket,This gainsboro jacket for women is perfect for...,apparel|jacket


## description, single_category 를 한국어로 번역

In [5]:
%%time

import pandas as pd

# df = pd.read_csv("amazon_faq_en.csv")
# print(df.columns)

def translate_df(df, target_col, new_col, length):
    
    def translate(df, length):
        text = df[target_col]
        result = translate_client.translate_text(Text=text, 
                SourceLanguageCode="en", TargetLanguageCode="ko")
        result = result['TranslatedText']
        result = result[0:length]
        return result
    df[new_col] = df.apply(translate, length=length, axis=1)
    
    return df

# token_length = 200 # 200 --> Error
token_length = 1200

df_ko = translate_df(df_en, target_col='description', new_col='description_ko', length = token_length)
df_ko = translate_df(df_en, target_col='single_category', new_col='single_category_ko', length = token_length)


CPU times: user 6.19 s, sys: 301 ms, total: 6.49 s
Wall time: 1min 16s


## single_category_ko 가 size_limit (예: 30) 이상 있는 것만 유지

In [6]:
pd.set_option('display.max_rows', 100)

def filter_dataset(df, size_limit=10):
    '''
    size_limit 이상인 카테고리만을 추출 함.
    '''
    print("Original shape: ", df.shape)    
    stat = df.groupby('single_category_ko').count()
    stat = pd.DataFrame(stat)

#    stat = stat.sort_values(by='single_category', ascending=False)
    stat = stat[stat.single_category > size_limit]
    # print("stat columns: ", stat.columns)    
    # print("single_category_ko: ", stat.index)    
    
    stat = df[df.single_category_ko.isin(stat.index)]
    print("new_df shape: ", stat.shape)    
    
    return stat

size_limit = 30
filter_df = filter_dataset(df_ko, size_limit = size_limit)
filter_df.head()

Original shape:  (2465, 6)
new_df shape:  (1871, 6)


,id,name,description,single_category,description_ko,single_category_ko
0,e1669081-8ffc-4dec-97a6-e9176d7f6651,Sans Pareil Scarf,Sans pareil scarf for women,apparel|scarf,여성용 산스 파레일 스카프,의복|스카프
1,cfafd627-7d6b-43a5-be05-4c7937be417d,Chef Knife,A must-have for your kitchen,housewares|kitchen,주방을 위한 머스트해브 아이템,가정용품|부엌
2,6e6ad102-7510-4a02-b8ce-5a0cd6f431d1,Gainsboro Jacket,This gainsboro jacket for women is perfect for...,apparel|jacket,이 여성용 게인즈버러 재킷은 도시에 딱 맞습니다.,의복|재킷
4,5cb18925-3a3c-4867-8f1c-46efd7eba067,Spiffy Sandals,This spiffy pair of sandals for woman is perfe...,footwear|sandals,이 푹신한 여성용 샌들은 여름에 딱 맞습니다.,신발|샌들
5,91cc9fa1-d8e9-46ae-9c8d-86264de2c6cc,Stylish Ceramic Bowl,This stylish ceramic bowl is a must-have,housewares|bowls,이 세련된 도자기 그릇은 필수품입니다.,가정용품|그릇


# 3. Train, Test 데이터 셋으로 층화 분리

In [8]:
from sklearn.model_selection import train_test_split

import numpy as np

def split_stratify_dataset(raw, test_size=0.2):
    X, y = raw.description_ko, raw.single_category_ko
    # 층화 데이터셋 분리
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, stratify=y, random_state=42)

    # 각 데이터셋의 클래스 비율 확인
    unique_train, counts_train = np.unique(y_train, return_counts=True)
    unique_test, counts_test = np.unique(y_test, return_counts=True)

    print("Train set class distribution:")
    print(dict(zip(unique_train, counts_train)))

    print("\nTest set class distribution:")
    print(dict(zip(unique_test, counts_test)))
    
    train_df = pd.DataFrame({'description_ko': X_train, 'single_category_ko': y_train}).reset_index(drop=True)
    test_df = pd.DataFrame({'description_ko': X_test, 'single_category_ko': y_test}).reset_index(drop=True)    
    
    return train_df, test_df

train_df, test_df = split_stratify_dataset(filter_df, test_size=0.1)

Train set class distribution:
{'가구|소파': 44, '가구|옷장': 29, '가구|의자': 65, '가구|테이블': 83, '가정용품|그릇': 49, '가정용품|부엌': 144, '계절|발렌타인': 33, '계절|할로윈': 41, '계절의|부활절': 38, '계절의|크리스마스': 83, '계측기|키': 30, '보석|팔찌': 33, '뷰티|그루밍': 37, '식료품|고기': 28, '식료품|빵집': 49, '식료품|야채': 37, '신발|샌들': 28, '신발|스니커즈': 43, '신발|정장': 71, '악기|타악기': 36, '악기|현악기': 55, '액세서리|배낭': 43, '액세서리|벨트': 34, '액세서리|안경': 53, '액세서리|핸드백': 34, '야외|낚시': 33, '의류|셔츠': 60, '의복|스카프': 48, '의복|재킷': 73, '전자제품|카메라': 32, '플로랄|식물': 38, '플로랄|어레인지먼트': 32, '홈 데코|데코레이션': 58, '홈 데코|조명': 48, '홈 데코|쿠션': 41}

Test set class distribution:
{'가구|소파': 5, '가구|옷장': 3, '가구|의자': 7, '가구|테이블': 9, '가정용품|그릇': 5, '가정용품|부엌': 16, '계절|발렌타인': 4, '계절|할로윈': 5, '계절의|부활절': 4, '계절의|크리스마스': 9, '계측기|키': 3, '보석|팔찌': 4, '뷰티|그루밍': 4, '식료품|고기': 3, '식료품|빵집': 5, '식료품|야채': 4, '신발|샌들': 3, '신발|스니커즈': 5, '신발|정장': 8, '악기|타악기': 4, '악기|현악기': 6, '액세서리|배낭': 5, '액세서리|벨트': 4, '액세서리|안경': 6, '액세서리|핸드백': 4, '야외|낚시': 4, '의류|셔츠': 7, '의복|스카프': 5, '의복|재킷': 8, '전자제품|카메라': 4, '플로랄|식물': 4, '플로랄|어레인지먼트': 4, '홈 데코|

In [11]:
train_df.head(5)

,description_ko,single_category_ko
0,특별한 날을 위한 머스트해브 아이템,계절|발렌타인
1,이 멋진 옷장에는 모든 물건을 정리할 수 있는 두 개의 서랍이 있습니다.,가구|옷장
2,이 맛있고 다원적인 채소로 식료품 저장실에 잘 비축해 두세요,식료품|야채
3,이 유리 꽃병은 모두를 기쁘게 할 것입니다,홈 데코|데코레이션
4,이 카메라는 특별한 순간을 포착하기에 완벽합니다.,전자제품|카메라


In [12]:
test_df.head(50)

,description_ko,single_category_ko
0,여성용 핑크 백팩,액세서리|배낭
1,여성용 블랙 포멀 슈즈,신발|정장
2,이 세라믹 꽃병은 홈 데코를 위한 머스트해브 아이템입니다.,홈 데코|데코레이션
3,멋진 여성용 캐주얼 재킷,의복|재킷
4,이 페루 오렌지 드레시 핸드백은 흠잡을 데 없습니다,액세서리|핸드백
5,이 업라이트 피아노는 가장 까다로운 뮤지션도 만족시킬 것입니다.,계측기|키
6,이 일렉트릭 기타는 가장 까다로운 뮤지션을 기쁘게 할 것입니다,악기|현악기
7,이 멋진 남성용 안경은 타의 추종을 불허합니다.,액세서리|안경
8,이 타의 추종을 불허하는 옷장에는 모든 물건을 정리할 수 있는 많은 서랍이 있습니다.,가구|옷장
9,우리 소고기는 농부들이 지속 가능한 방식으로 기르고 있습니다.,식료품|고기


# 4. JSON 파일 생성

In [13]:
def create_json_file(df, file_name):
    df.to_json(file_name, orient='records', force_ascii=False)
    print(f"{file_name} is generated")

create_json_file(train_df, file_name='retail_demo_train.json')    
create_json_file(test_df, file_name='retail_demo_test.json')    

retail_demo_train.json is generated
retail_demo_test.json is generated
